In [ ]:
orgn_arr= np.nan_to_num(jij_data_array) 
        avrg= np.nan_to_num(jij_data_array)
        ## As this is the ExtractJijData array_data along the x-aixs as well as y-axis are equal length
        for i in range(np.shape(jij_data_array)[1]):
            avrg[:,i] += orgn_arr[i,:] 